In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.special import expit

In [12]:
def abrirImagenesEscaladas( carpeta, escala=200 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [13]:
def balancear_datos(imagenes_entrenamiento):
    imagenes_entrenamiento_balanceadas = abrirImagenesEscaladas(imagenes_entrenamiento)
    return imagenes_entrenamiento_balanceadas

In [14]:
imagenes_sanos = np.array(balancear_datos("train_mini/sano")) #len = 349
imagenes_neum = np.array(balancear_datos("train_mini/neumonia")) #len = 349


In [29]:
# Definir el número de iteraciones y la tasa de aprendizaje
MAX_ITER = 1000
TOLERANCIA = 0.0001

In [8]:
def f(x,w,b):
    return (np.tanh(np.dot(x,w)+b)+1)/2

In [16]:
def L(x,d,w,b):
    y_p = f(x,w,b)
    return (y_p-d)**2

In [61]:
def normalizar(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [20]:
# Derivadas de L(w, b)
def dW(x,d,w,b):
    y_p = f(x,w,b)
    return (y_p-d)*(1-np.tanh(np.dot(x,w)+b)**2) * x

def dB(x,d,w,b):
    y_p = f(x,w,b)
    return (y_p-d)*(1-np.tanh(np.dot(x,w)+b)**2)

In [118]:
# Función de búsqueda del mínimo de L(w, b)
def gradiente_descendente(x_sanos,x_enfermos,alpha):
    # Valores iniciales de w y b
    w_t = np.zeros(x_sanos.shape[1]) #x_sanos y x_enfermos del mismo tamaño
    b_t = 0

    # Mientras no hayamos llegado al maximo de iteraciones
    iter = 0
    error_anterior = float('inf')

    while iter <= MAX_ITER:
        # Computamos siguientes w y b a partir de las derivadas de la función
        grad_w = 0
        grad_b = 0
        error = 0

        
        d_sanos=0 #diagnostico para el conjunto de imagenes sin neumonía
        for i in range(len(x_sanos)):  
            grad_w += dW(x_sanos[i],d_sanos,w_t,b_t)
            grad_b += dB(x_sanos[i],d_sanos,w_t,b_t)
            error += L(x_sanos[i],d_sanos,w_t,b_t)
        
        d_neum=1 #diagnostico para el conjunto de imagenes con neumonía
        for i in range(len(x_enfermos)):
            grad_w += dW(x_enfermos[i],d_neum,w_t,b_t)
            grad_b += dB(x_enfermos[i],d_neum,w_t,b_t)
            error+=L(x_enfermos[i],d_neum,w_t,b_t)

        #se toma promedio porque de otra manera los numeros dan muy grandes y se entorpece la solucion
        grad_w /= (len(x_sanos) + len(x_enfermos))
        grad_b /= (len(x_sanos) + len(x_enfermos))
        error /= (len(x_sanos) + len(x_enfermos))
        
        print(f'iteracion: {iter}, error: {error}')

        w_tsig = w_t - alpha * grad_w
        b_tsig = b_t - alpha * grad_b

        # Chequeamos si ya alcanzamos la convergencia
        if abs(error_anterior - error) < TOLERANCIA:
            break

        # Preparamos la siguiente iteración
        w_t = w_tsig
        b_t = b_tsig
        error_anterior = error
        iter = iter + 1

    return w_t, b_t

In [123]:
alfa = 0.001
w,b = gradiente_descendente(imagenes_sanos,imagenes_neum,alfa)

iteracion: 0, error: 0.25
iteracion: 1, error: 0.23977535590160454
iteracion: 2, error: 0.25015373134683916
iteracion: 3, error: 0.40787545422639376
iteracion: 4, error: 0.23770169451544426
iteracion: 5, error: 0.40839382916960676
iteracion: 6, error: 0.22162923647084234
iteracion: 7, error: 0.3899380608536261
iteracion: 8, error: 0.2724064233037038
iteracion: 9, error: 0.45879444546387915
iteracion: 10, error: 0.3692479002448059
iteracion: 11, error: 0.3350697993618571
iteracion: 12, error: 0.4214202697489124
iteracion: 13, error: 0.17860000454968555
iteracion: 14, error: 0.25010464255742426
iteracion: 15, error: 0.4589017266230058
iteracion: 16, error: 0.3706097799958415
iteracion: 17, error: 0.2933033200025059
iteracion: 18, error: 0.4469363366533755
iteracion: 19, error: 0.29611295672915416
iteracion: 20, error: 0.43814241997170206
iteracion: 21, error: 0.21369052518679965
iteracion: 22, error: 0.44778161884935297
iteracion: 23, error: 0.3034988389186948
iteracion: 24, error: 0.419

In [124]:
x_train_sanos = np.array(balancear_datos("test/NORMAL"))
x_train_neum = np.array(balancear_datos("test/PNEUMONIA"))
w_train, b_train = gradiente_descendente(x_train_sanos, x_train_neum, 0.001)


iteracion: 0, error: 0.25
iteracion: 1, error: 0.3200281523575595
iteracion: 2, error: 0.22740557923306928
iteracion: 3, error: 0.23131682342312368
iteracion: 4, error: 0.31089672735176105
iteracion: 5, error: 0.21377317118174624
iteracion: 6, error: 0.2483500238431357
iteracion: 7, error: 0.3924581815233291
iteracion: 8, error: 0.36465386780034476
iteracion: 9, error: 0.35978016010417035
iteracion: 10, error: 0.3485016396325474
iteracion: 11, error: 0.310709123518299
iteracion: 12, error: 0.19327034798825332
iteracion: 13, error: 0.205296015071784
iteracion: 14, error: 0.3679593133521494
iteracion: 15, error: 0.3662546063477467
iteracion: 16, error: 0.3628151916698052
iteracion: 17, error: 0.3557708156158767
iteracion: 18, error: 0.33657923025564995
iteracion: 19, error: 0.25453731727453804
iteracion: 20, error: 0.36413715538540853
iteracion: 21, error: 0.36543014657249057
iteracion: 22, error: 0.361225832409013
iteracion: 23, error: 0.3519463327961749
iteracion: 24, error: 0.32307673

In [132]:
test_sanos = np.round(f(x_train_sanos, w_train, b_train))

suma_errores = 0
for i in range(len(test_sanos)):
    if test_sanos[i] == 1:
        suma_errores +=1

print(f'Diagnosticos a partir de descenso por gradiente: {test_sanos}; \nErrores: {suma_errores} de {len(test_sanos)}')

Diagnosticos a partir de descenso por gradiente: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]; 
Errores: 49 de 234


In [131]:
test_neumonia = np.round(f(x_train_neum, w_train, b_train))


suma_errores = 0
for i in range(len(test_neumonia)):
    if test_neumonia[i] == 0:
        suma_errores +=1

print(f'Diagnosticos a partir de descenso por gradiente: {test_neumonia}; \nErrores: {suma_errores} de {len(test_neumonia)}')

Diagnosticos a partir de descenso por gradiente: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1